# Part 1 - Data Cleaning

In [1]:
# import libraries
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup as bs
import json
import requests
%matplotlib inline

In [2]:
# import data
housing_df=pd.read_csv("../datasets/train.csv")
housing_df.head()

C:\Users\songk\AppData\Local\Temp\ipykernel_29608\1090650860.py:2: DtypeWarning: Columns (41) have mixed types. Specify dtype option on import or set low_memory=False.
  housing_df=pd.read_csv("../datasets/train.csv")


,id,Tranc_YearMonth,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,...,vacancy,pri_sch_affiliation,pri_sch_latitude,pri_sch_longitude,sec_sch_nearest_dist,sec_sch_name,cutoff_point,affiliation,sec_sch_latitude,sec_sch_longitude
0,88471,2016-05,KALLANG/WHAMPOA,4 ROOM,3B,UPP BOON KENG RD,10 TO 12,90.0,Model A,2006,...,78,1,1.317659,103.882504,1138.633422,Geylang Methodist School,224,0,1.317659,103.882504
1,122598,2012-07,BISHAN,5 ROOM,153,BISHAN ST 13,07 TO 09,130.0,Improved,1987,...,45,1,1.349783,103.854529,447.894399,Kuo Chuan Presbyterian Secondary School,232,0,1.350110,103.854892
2,170897,2013-07,BUKIT BATOK,EXECUTIVE,289B,BT BATOK ST 25,13 TO 15,144.0,Apartment,1997,...,39,0,1.345245,103.756265,180.074558,Yusof Ishak Secondary School,188,0,1.342334,103.760013
3,86070,2012-04,BISHAN,4 ROOM,232,BISHAN ST 22,01 TO 05,103.0,Model A,1992,...,20,1,1.354789,103.844934,389.515528,Catholic High School,253,1,1.354789,103.844934
4,153632,2017-12,YISHUN,4 ROOM,876,YISHUN ST 81,01 TO 03,83.0,Simplified,1987,...,74,0,1.416280,103.838798,312.025435,Orchid Park Secondary School,208,0,1.414888,103.838335


We observe column 42 (postal) has mixed data types. Here we standardise the datatype in this column by setting as string.

In [3]:
housing_df.columns[41]

'postal'

In [4]:
# convert postal to string
housing_df['postal']=housing_df['postal'].map(lambda x: str(x))

In [5]:
housing_df.shape

(150634, 77)

In [6]:
housing_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150634 entries, 0 to 150633
Data columns (total 77 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   id                         150634 non-null  int64  
 1   Tranc_YearMonth            150634 non-null  object 
 2   town                       150634 non-null  object 
 3   flat_type                  150634 non-null  object 
 4   block                      150634 non-null  object 
 5   street_name                150634 non-null  object 
 6   storey_range               150634 non-null  object 
 7   floor_area_sqm             150634 non-null  float64
 8   flat_model                 150634 non-null  object 
 9   lease_commence_date        150634 non-null  int64  
 10  resale_price               150634 non-null  float64
 11  Tranc_Year                 150634 non-null  int64  
 12  Tranc_Month                150634 non-null  int64  
 13  mid_storey                 15

In [7]:
#rename all columns using lower caps
housing_df.columns=housing_df.columns.str.strip().str.lower() 

In [8]:
# rename to sec sch afiliation

housing_df.rename(columns={'affiliation':'sec_sch_affiliation'},inplace=True)

In [9]:
# check for nulls in each column
housing_df[housing_df.columns[housing_df.isnull().any()]].isnull().sum()

mall_nearest_distance      829
mall_within_500m         92789
mall_within_1km          25426
mall_within_2km           1940
hawker_within_500m       97390
hawker_within_1km        60868
hawker_within_2km        29202
dtype: int64

Out of all 76 columns in dataset, we have 7 columns containing nulls as shown above.

### Accounting for null values in hawker quantity columns

As there are no missing values for hawker nearest distance, a possibility might be that the hawker nearest distance in specific rows has exceeded 500m/1km/2km, hence resulting in hawker null values. This can be easily validated by finding the minimum distance for specific rows containing nulls, as shown below.

In [10]:
#check if there is any discrepancy in minimum distance larger than 500m/1km/2km for rows with missing hawker quantity

print(f"Minimium distance greater than 500m: \
{round(housing_df[housing_df['hawker_within_500m'].isnull()]['hawker_nearest_distance'].min(),3)}m")

print(f"Minimium distance greater than 1km: \
{round(housing_df[housing_df['hawker_within_1km'].isnull()]['hawker_nearest_distance'].min(),3)}m")

print(f"Minimium distance greater than 2km: \
{round(housing_df[housing_df['hawker_within_2km'].isnull()]['hawker_nearest_distance'].min(),3)}m")

Minimium distance greater than 500m: 500.209m
Minimium distance greater than 1km: 1000.074m
Minimium distance greater than 2km: 2000.872m


For the rows with missing hawker value, we see that the available minimum distance greater than 500m, 1km and 2km is 500.209m, 1000.074m and 2000.872m respectively. This implies that the nulls in each of these 3 columns can be set to 0, since the minimum distance present is larger than the specified distance.

In [11]:
# replace all missing hawker value to 0
housing_df.loc[housing_df['hawker_within_500m'].isnull(),'hawker_within_500m']=0
housing_df.loc[housing_df['hawker_within_1km'].isnull(),'hawker_within_1km']=0
housing_df.loc[housing_df['hawker_within_2km'].isnull(),'hawker_within_2km']=0

### Accounting for null values in mall quantity columns

As we have 829 rows with missing mall nearest distance, firstly we need to address the nulls in this column before filling in the missing mall quantity values. The procedure to find the nearest mall distance in these rows is shown below.

1. Retrieve all malls in Singapore, this can be done by webscraping from Wikipedia.
2. Use OneMap API to locate the latitude/longitude of the malls.
3. For each housing unit, calculate distance between itself and each mall using Haversine formula. Locate the minimum mall distance and replace the null value.

An important point to note is that the rows with missing mall distance are identified through their row index in the dataset.

In [12]:
# Retrieve all malls in Singapore from Wikipedia
url='https://en.wikipedia.org/wiki/List_of_shopping_malls_in_Singapore'
response = requests.get(url)
print('Status Code: ',response.status_code)
html = response.content
soup = bs(html, 'lxml')
soup

malls = soup.find_all("div",{'class':'div-col'})
list_of_malls=[]
for i in range(len(malls)):
    list_of_malls+=[x.text for x in malls[i].find_all("li")]
list_of_malls=[x[:x.find("[")] if x.find("[")!=-1 else x for x in list_of_malls]
list_of_malls

Status Code:  200


['100 AM',
 '313@Somerset',
 'Aperia',
 'Balestier Hill Shopping Centre',
 'Bugis Cube',
 'Bugis Junction',
 'Bugis+',
 'Capitol Piazza',
 'Cathay Cineleisure Orchard',
 'Clarke Quay Central',
 'The Centrepoint',
 'City Square Mall',
 'City Gate Mall',
 'CityLink Mall',
 'Duo',
 'Far East Plaza',
 'Funan',
 'Great World City',
 'HDB Hub',
 'Holland Village Shopping Mall',
 'ION Orchard',
 'Junction 8',
 'Knightsbridge',
 'Liat Towers',
 'Lucky Plaza',
 'Marina Bay Sands',
 'The Shoppes at Marina Bay Sands',
 'Marina Bay Link Mall',
 'Marina Square',
 'Millenia Walk',
 'Mustafa Shopping Centre',
 'Ngee Ann City',
 'Orchard Central',
 'Orchard Gateway',
 'Orchard Plaza',
 'Midpoint Orchard',
 'Palais Renaissance',
 "People's Park Centre",
 "People's Park Complex",
 'Plaza Singapura',
 'PoMo',
 'Raffles City',
 'Scotts Square',
 'Shaw House and Centre',
 'Sim Lim Square',
 'Singapore Shopping Centre',
 'The South Beach',
 'Square 2',
 'Sunshine Plaza',
 'Suntec City',
 'Tanglin Mall',
 'T

In [13]:
# Obtaining Mall Coordinates in Singapore from OneMAP API
list_of_malls
mall_name = []
mall_lat = []
mall_long = []

for i in range(0, len(list_of_malls)):
    query_address = list_of_malls[i]
    query_string = 'https://developers.onemap.sg/commonapi/search?searchVal='+str(query_address)+'&returnGeom=Y&getAddrDetails=Y'
    resp = requests.get(query_string)
    data_mall=json.loads(resp.content)
    
    if data_mall['found'] != 0:
        mall_name.append(query_address)
        mall_lat.append(float(data_mall["results"][0]["LATITUDE"]))
        mall_long.append(float(data_mall["results"][0]["LONGITUDE"]))

        print (str(query_address)+", Latitude: "+data_mall['results'][0]['LATITUDE'] +" Longitude: "+data_mall['results'][0]['LONGITUDE'])

    else:
        print (f"No result for {list_of_malls[i]}")

100 AM, Latitude: 1.27458821795426 Longitude: 103.84347073661
313@Somerset, Latitude: 1.30097808212211 Longitude: 103.838350465208
Aperia, Latitude: 1.31047040225956 Longitude: 103.864287497404
Balestier Hill Shopping Centre, Latitude: 1.32557113096136 Longitude: 103.842493835321
Bugis Cube, Latitude: 1.29819500487878 Longitude: 103.855655544739
Bugis Junction, Latitude: 1.2991371723215 Longitude: 103.855450325604
Bugis+, Latitude: 1.30095171530648 Longitude: 103.855172625542
Capitol Piazza, Latitude: 1.29306335877347 Longitude: 103.851293463737
Cathay Cineleisure Orchard, Latitude: 1.30153093362853 Longitude: 103.836421018025
No result for Clarke Quay Central
The Centrepoint, Latitude: 1.30145045537088 Longitude: 103.840034074858
City Square Mall, Latitude: 1.31135169933195 Longitude: 103.856724057693
No result for City Gate Mall
CityLink Mall, Latitude: 1.29260349413333 Longitude: 103.854316064564
Duo, Latitude: 1.29953434891664 Longitude: 103.85840168774
Far East Plaza, Latitude: 1.

We observe 9 malls that were unable to retrieve latitude and longitude values automatically, this missing latitude/longitude information will be added manually by looking up online or refining the search term in OneMAP API.

In [14]:
#add in latitude and longitude manually as search criteria not exactly matching between Wikipedia and OneMAP API
missing_malls={
    'Mall': ['Clarke Quay Central','City Gate Mall','Holland Village Shopping Mall','Mustafa Shopping Centre','PoMo','Shaw House and Centre','KINEX','Paya Lebar Quarter (PLQ)','OD Mall'],
    'Latitude': [1.2882413,1.30231590504573,1.31027747574118,1.31011213190394,1.300058,1.305512,1.31479,1.317369,1.3380],
    'Longitude': [103.846430401652,103.862331661034,103.795371163103,103.855290873926,103.849079,103.831755,103.89464,103.893266,103.7934]
}

mall_name += missing_malls['Mall']
mall_lat += missing_malls['Latitude']
mall_long += missing_malls['Longitude']

In [15]:
#calculate distance between 2 points using Haversine formula

from math import radians, cos, sin, asin, sqrt
def dist(lat1, lon1, lat2, lon2):
     
    # The math module contains a function named radians which converts from degrees to radians.
    lon1 = radians(lon1)
    lon2 = radians(lon2)
    lat1 = radians(lat1)
    lat2 = radians(lat2)
      
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
 
    c = 2 * asin(sqrt(a))
    
    # Radius of earth in kilometers. Use 3956 for miles
    r = 6371
      
    # calculate the result
    return(c *r*1000)

In [16]:
missing_lat=[x[42] for x in housing_df[housing_df['mall_nearest_distance'].isnull()].values]
missing_long=[x[43] for x in housing_df[housing_df['mall_nearest_distance'].isnull()].values]

In [17]:
row_index=housing_df[housing_df['mall_nearest_distance'].isnull()]['mall_nearest_distance'].index

In [18]:
#row index where mall nearest distance is null
print(list(row_index))

[75, 321, 478, 643, 691, 835, 990, 1092, 1160, 1292, 1575, 1593, 1766, 2222, 2517, 2524, 3094, 3165, 3333, 3399, 3472, 4258, 4357, 4436, 4492, 4680, 4912, 4970, 5364, 5375, 5723, 5737, 5898, 6019, 6152, 6771, 6800, 6860, 7263, 7346, 7381, 7531, 7637, 8110, 8148, 8434, 8528, 8616, 8915, 9151, 9156, 9691, 9899, 10108, 10404, 10461, 10465, 10574, 10768, 10820, 11221, 11259, 11280, 11290, 11307, 11516, 11776, 11787, 11853, 12297, 12410, 13038, 13042, 13077, 13306, 13332, 13390, 13470, 13616, 13967, 14203, 14298, 14422, 14471, 14659, 15315, 15471, 15612, 15785, 16297, 16447, 16503, 16668, 16788, 16806, 17282, 17705, 17863, 17987, 18034, 18164, 18172, 18272, 18614, 18696, 18848, 19078, 19292, 19310, 19458, 19469, 19628, 19658, 19828, 19852, 20115, 20234, 20256, 20516, 21421, 21472, 22427, 22704, 22705, 22837, 22963, 23364, 23518, 23592, 23902, 23976, 24023, 24054, 24056, 24545, 25651, 25755, 25853, 25928, 26237, 26716, 26944, 27322, 27505, 27513, 27619, 28074, 28213, 28273, 28319, 28472, 285

In [19]:
def mall_nearest_dist(latitude,longitude):
    distance=[]
    for i in range(len(latitude)):
        x=[]
        for j in range(len(mall_name)):
            x.append(dist(latitude[i],longitude[i],mall_lat[j],mall_long[j]))
        distance.append(pd.DataFrame(x).min().values[0])
    return distance

distance=pd.DataFrame(mall_nearest_dist(missing_lat,missing_long)).rename(columns={0:'mall_nearest_distance'})
distance.rename(index={x:row_index[x] for x in range(len(distance))},inplace=True)

def mall_dist(x):
    if pd.isna(housing_df['mall_nearest_distance'][x]):
        return distance['mall_nearest_distance'][x]
    else:
        pass

In [20]:
distance

,mall_nearest_distance
75,713.252783
321,364.332266
478,1249.235253
643,452.221992
691,125.537430
...,...
150296,615.808682
150388,422.996655
150394,607.721286
150479,516.718308


In [21]:
# if null detected in mall nearest distance, it will be replaced by a value from the distance dataframe (with matching index, 
# since the row index with nulls has already been established)
for i in range(len(housing_df)):
    if i in row_index.values:
        housing_df.loc[i,'mall_nearest_distance']=mall_dist(i)

In [22]:
housing_df.loc[row_index,['mall_nearest_distance','mall_within_500m','mall_within_1km','mall_within_2km']].isnull().sum()

mall_nearest_distance      0
mall_within_500m         829
mall_within_1km          829
mall_within_2km          829
dtype: int64

These 829 rows already have a nearest mall distance, as expected they do not have a mall quantity within 500m/1km/2km. We proceed to calculate the 3 values for each row.

In [23]:
def mall_500m(latitude,longitude):
    n_malls_500m=[]
    for i in range(len(latitude)):
        x=[]
        for j in range(len(mall_name)):
            x.append(dist(latitude[i],longitude[i],mall_lat[j],mall_long[j]))
        x=pd.DataFrame(x).rename(columns={0:'dist'})
        n_malls_500m.append(x[x['dist']<=500].count().values[0])
    return n_malls_500m

n_malls_500m=pd.DataFrame(mall_500m(missing_lat,missing_long)).rename(columns={0:'mall_within_500m'})
n_malls_500m.rename(index={x:row_index[x] for x in range(len(distance))},inplace=True)

In [24]:
n_malls_500m

,mall_within_500m
75,0
321,1
478,0
643,1
691,3
...,...
150296,0
150388,1
150394,0
150479,0


In [25]:
# calculate the number of malls within 500m if the row index previously had a null value for mall_nearest_distance
for i in range(len(housing_df)):
    if i in row_index.values:
        housing_df.loc[i,'mall_within_500m']=n_malls_500m['mall_within_500m'][i]

In [26]:
def mall_1km(latitude,longitude):
    n_malls_1km=[]
    for i in range(len(latitude)):
        x=[]
        for j in range(len(mall_name)):
            x.append(dist(latitude[i],longitude[i],mall_lat[j],mall_long[j]))
        x=pd.DataFrame(x).rename(columns={0:'dist'})
        n_malls_1km.append(x[x['dist']<=1000].count().values[0])
    return n_malls_1km

n_malls_1km=pd.DataFrame(mall_1km(missing_lat,missing_long)).rename(columns={0:'mall_within_1km'})
n_malls_1km.rename(index={x:row_index[x] for x in range(len(distance))},inplace=True)

In [27]:
n_malls_1km

,mall_within_1km
75,1
321,3
478,0
643,2
691,3
...,...
150296,1
150388,2
150394,2
150479,2


In [28]:
for i in range(len(housing_df)):
    if i in row_index.values:
        housing_df.loc[i,'mall_within_1km']=n_malls_1km['mall_within_1km'][i]

In [ ]:
def mall_2km(latitude,longitude):
    n_malls_2km=[]
    for i in range(len(latitude)):
        x=[]
        for j in range(len(mall_name)):
            #x.append(dist(missing.values[0][2],missing.values[0][3],mall_lat[i],mall_long[i]))
            x.append(dist(latitude[i],longitude[i],mall_lat[j],mall_long[j]))
        x=pd.DataFrame(x).rename(columns={0:'dist'})
        n_malls_2km.append(x[x['dist']<=2000].count().values[0])
    return n_malls_2km

n_malls_2km=pd.DataFrame(mall_2km(missing_lat,missing_long)).rename(columns={0:'mall_within_2km'})
n_malls_2km.rename(index={x:row_index[x] for x in range(len(distance))},inplace=True)

In [ ]:
n_malls_2km

In [ ]:
for i in range(len(housing_df)):
    if i in row_index.values:
        housing_df.loc[i,'mall_within_2km']=n_malls_2km['mall_within_2km'][i]

In [ ]:
housing_df.loc[:,['mall_nearest_distance','mall_within_500m','mall_within_1km','mall_within_2km']].isnull().sum()

As there are no missing values for mall nearest distance, it is highly likely that the mall nearest distance in specific rows has exceeded 500m/1km/2km, hence resulting in mall null values. This can be easily validated by finding the minimum distance for specific rows containing nulls, as shown below.

In [ ]:
#check if there is any discrepancy in minimum mall distance larger than 500m/1km/2km for rows with missing mall count
print(f"Minimium mall distance greater than 500m: \
{round(housing_df[housing_df['mall_within_500m'].isnull()]['mall_nearest_distance'].min(),3)}m")

print(f"Minimium mall distance greater than 1km: \
{round(housing_df[housing_df['mall_within_1km'].isnull()]['mall_nearest_distance'].min(),3)}m")

print(f"Minimium mall distance greater than 2km: \
{round(housing_df[housing_df['mall_within_2km'].isnull()]['mall_nearest_distance'].min(),3)}m")

For the rows with missing mall value, we see that the available minimum distance greater than 500m, 1km and 2km is 500.056m, 1000.023m and 2000.176m respectively. This implies that the nulls in each of these 3 columns can be set to 0, since the minimum distance present is larger than the specified distance.

In [ ]:
housing_df.loc[housing_df['mall_within_500m'].isnull(),'mall_within_500m']=0
housing_df.loc[housing_df['mall_within_1km'].isnull(),'mall_within_1km']=0
housing_df.loc[housing_df['mall_within_2km'].isnull(),'mall_within_2km']=0

Lastly, we convert the values in specific columns to appropriate data types. This is done at the very end as there were nulls in some of the columns and datatype conversion was not possible.

In [ ]:
#change specific columns in dataset to appropriate data types
housing_df['id']=housing_df['id'].map(lambda x: str(x))
housing_df['mall_within_500m']=housing_df['mall_within_500m'].map(lambda x: int(x))
housing_df['mall_within_1km']=housing_df['mall_within_1km'].map(lambda x: int(x))
housing_df['mall_within_2km']=housing_df['mall_within_2km'].map(lambda x: int(x))
housing_df['hawker_within_500m']=housing_df['hawker_within_500m'].map(lambda x: int(x))
housing_df['hawker_within_1km']=housing_df['hawker_within_1km'].map(lambda x: int(x))
housing_df['hawker_within_2km']=housing_df['hawker_within_2km'].map(lambda x: int(x))

In [ ]:
# to verify if there are any missing values in each column
housing_df.isnull().sum().sum()

In [ ]:
# to check if there are any identical rows
housing_df.duplicated().sum()

We have verified that the housing dataset does not contain any nulls and no duplicated rows are seen as well.

In [ ]:
#save and export data for further analysis
housing_df.to_csv("../datasets/cleaned_housing_data.csv",index=False)